<div>
    <img style="float:right;" src="images/smi-logo.png"/>
    <div style="float:left;color:#58288C;"><h1>Datenanalyse und Datenmanagement</h1></div>
</div>

---
# Notebook II: Data Management
In diesem Notebook geht es um die Arbeit mit Datenbanken und APIs.

## Inhaltsverzeichnis

[1. Einführung in SQL mit Notebooks](#kapitel1)  
[2. Erstellen von Tabellen und Daten](#kapitel2)  
[3. Abfragen von Daten](#kapitel3)  
[4. Verbindung zu Datenbankservern herstellen](#kapitel4)  
[5. Verbindung zu APIs herstellen](#kapitel5)  
[6. Übungen Datenbank- und API-Abfragen](#kapitel6)  

---

## 1. Einführung Datenbanken in Notebooks<a id="kapitel1"/>

### 1.1. Jupyter Notebooks und SQL

Obwohl wir mit einem Python-Notebook arbeiten, können wir in einer Zelle temporär andere Sprachen verwenden. Hierzu müssen wir eine Notebook-Extension laden (in unserem Beispiel ```sql```).

In [42]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Ab jetzt können Zellen als SQL-Zellen gekennzeichnet werden:

In [ ]:
%sql -- einzeilige SQL-Anweisung

In [ ]:
%%sql

-- ganze Zelle ist SQL-Anweisung

### 1.2. SQLite

Unsere ersten Schritte machen wir mit einer Datenbank vom Typ SQLite (mehr Infos dazu [hier](https://www.sqlite.org/index.html)). SQLite Datenbanken liegen typischerweise als eine einzelne Datei vor und können so flexibel in Programme verschiedenster Sprachen eingebunden werden. Die Dokumentation findet sich [hier](https://sqlite.org/lang.html).

SQLite wird auf dessen Website so beschrieben:  
> SQLite [...] implements a small, fast, self-contained, high-reliability, full-featured, SQL database engine. SQLite is the most used database engine in the world. SQLite is built into all mobile phones and most computers and comes bundled inside countless other applications that people use every day.

---

In SQL-Zellen können wir mit einem URL (Uniform Resource Locator) ähnlich dem Webbrowser eine Verbindung zu Datenbanken herstellen. Der Aufbau ist im Allgemeinen wie folgend: 
> _Datenbanktyp_://_Username_:_Password_@_Servername_/_Datenbankname_ 

Für SQlite-Datenbanken im Dateisystem des Notebooks entfallen Username, Password und Servername (daher drei "/"), der Datenbankname ist der Dateiname:

> _sqlite:///**pfad/zur/datenbankdatei**_

In [ ]:
%%sql 

sqlite:///my-database.db

## 2. Erstellen von Tabellen und Daten <a id="kapitel2"/>
SQL-Befehle sind etwas anders aufgebaut als Python-Befehle. Für unsere Zwecke beachte zwei Dinge:
- Ein Befehl ist mit ```;``` abzuschließen, sonst wird er nicht ausgeführt
- Kommentare sind mit ```--```einzuleiten statt mit ```#``` 

Nachfolgend erzeugen wir eine Datenstruktur, wie sie in folgendem Klassendiagramm abgebildet ist:  
<img style="width:450px;" src="images/02_class-diagram.png"/>

### 2.1. Erzeugen von Beispieltabellen und -daten
Erzeugen wir nun im ersten Schritt mit SQL die Tabelle "Mitarbeiter" mit dem Befehl ```CREATE TABLE```.  

Auf deinem Smartphone würden diese Kommandos beispielsweise beim ersten Start durch eine App ausgeführt.

In [ ]:
%%sql  

-- Falls es die Tabellen schon geben sollte (etwa weil du die Zelle schon einmal ausgeführt hast): Tabellen löschen
DROP TABLE IF EXISTS Mitarbeiter; 
DROP TABLE IF EXISTS Abteilungen; 

CREATE TABLE Abteilungen (
    abteilungs_nr int PRIMARY KEY,
    bezeichnung varchar(50)
);

CREATE TABLE Mitarbeiter (
    mitarbeiter_nr int PRIMARY KEY,
    abteilungs_nr int,                    -- hier haben wir die Id der Abteilung als Fremdschlüssel eingefügt
    vorname varchar(50),
    name varchar(50),
    FOREIGN KEY (abteilungs_nr) REFERENCES Abteilungen(abteilungs_nr)
    );

Nun fügen wir mit ```INSERT``` Datensätze hinzu:

In [ ]:
%%sql

INSERT INTO Abteilungen VALUES (1, "Einkauf");
INSERT INTO Abteilungen VALUES (2, "Produktion");
INSERT INTO Abteilungen VALUES (3, "Vertrieb");

INSERT INTO Mitarbeiter VALUES (1, 2, "Marco", "Lind");  -- id "1", Abteilungsnr. "2", Vorname "Marco", Nachname "Lind"
INSERT INTO Mitarbeiter VALUES (2, 2, "Peter", "Lind");
INSERT INTO Mitarbeiter VALUES (3, 3, "Sophia", "Lind");

### 2.2. Demo: Integritätsprüfung
Wenn wir nun das Überprüfen der Fremdschlüssel (foreign keys) scharf schalten, kann die Datenbank die Datenkonsistenz sicherstellen:  
- **Referenzielle Integrität**: Wir können keine Mitarbeiter mit ungültigen Abteilungs-Ids anlegen und keine Abteilungen löschen, denen noch Mitarbeiter zugeordnet sind.
- **Vermeiden von Redundanz**: Müssten wir zu jedem Mitarbeiter den Abteilungsnamen ausschreiben, wäre das Umbenennen von Abteilungen aufwändig - jeder Mitarbeiterdatensatz müsste geändert werden. 

Auch vermeiden wir so inkonsistente Schreibweisen der Abteilungsnamens und andere Fehlerquellen.  

In [ ]:
%%sql 
PRAGMA foreign_keys=1;                                          -- Überprüfung der Fremdschlüssel "scharf" schalten
INSERT INTO Mitarbeiter VALUES (4, 5, "Gerhard", "Lind");       -- schlägt fehl: Abteilungs_nr 5 gibt es nicht

---

## 3. Abfragen von Daten <a id="kapitel3"/>

### 3.1. Daten aus einzelnen Tabellen
Nun ist es naheliegend, diese Daten wieder Abfragen zu wollen. Dies geschieht mit dem ```SELECT``` Befehl. Eine Smartphone-App würde mittels SELECT  beispielsweise beim Start die benötigten Daten zur Anzeige aus dem Handyspeicher laden.

In [ ]:
%%sql

SELECT vorname, name FROM Mitarbeiter;

Anstelle einer Auflistung der Datenfelder können mit ```*``` alle Felder abgefragt werden.  
Mit dem Unterbefehl ```WHERE``` kannst du die Ergebnisdaten filtern, mit ```ORDER BY``` sortieren:

In [ ]:
%%sql

SELECT * FROM Mitarbeiter
WHERE abteilungs_nr = 2           -- nur aus Abteilung
ORDER BY vorname;                 -- alphabetisch nach Vorname sortieren

---
### <span style="color:#FF5D02;">Aufgabe: Überprüfe die Tabelle "Abteilungen"</span>
a) Lass die alle Datensätze der Tabelle "Abteilungen" mittels ```SELECT``` anzeigen.  
b) Sortiere alphabetisch nach Abteilungsname.


In [ ]:
%%sql



---

### 3.2. Ausblick: Abfrage aus verknüpften Tabellen
Nun wollen wir aber die Abteilungsnamen nicht manuell nachschlagen, sondern beide Tabelle in einer Abfrage zusammenführen.


In [ ]:
%%sql

SELECT mitarbeiter_nr, vorname, name, bezeichnung AS Abteilung            -- wir benennen hier die Spalte "bezeichnung" in "Abteilung" um
FROM Mitarbeiter, Abteilungen
WHERE Mitarbeiter.abteilungs_nr = Abteilungen.abteilungs_nr;              -- Die Abteilungsnr. aus Tabelle Mitarbeiter und Abteilung muss übereinstimmen

## 4. Daten von Datenbankservern abrufen <a id="kapitel4"/>

Im ersten Schritt müssen wir die ZIP-komprimierte Datenbankdatei auspacken.

In [ ]:
!unzip -o -d data data/smi-data.zip

Nun können wir eine Verbindung zu der SQLite-Datenbank in der Datei ```smi-data.db``` im Unterverzeichnis ```data``` herstellen.

In [43]:
%sql sqlite:///data/smi-data.db

SQLite hebt datenbankinterne Informationen in einer automatisch angelegten Tabelle ```sqlite_master``` an.  
Hier lässt sich beispielsweise abfragen, welche Tabellen es gibt.

In [44]:
%sql SELECT name FROM sqlite_master WHERE type="table"

 * sqlite:///data/smi-data.db
   sqlite:///my-database.db
Done.


name
okcupid
credit_ger
housing_ger
telco_churn


### Arbeit mit dem OkCupid-Datensatz
Nun können wir die bekannten Befehle nutzen, um Daten abzufragen und zu manipulieren.  
In diesem Beispiel siehst du, wie ein Datensatz der Dating-Plattform OKCupid nach Profilen durchsucht wird, die bestimmten Suchkriterien entsprechen:

In [ ]:
%%sql 

SELECT *
FROM okcupid
WHERE (drugs="never") AND (income > 40000) AND (age > 20) AND (age < 30)
  AND (sex="f") AND (orientation="straight") AND (pets="likes dogs")
LIMIT 20;

---
### <span style="color:#FF5D02;">Aufgabe: Abfragen, filtern und sortieren mit SQL</span>
Verändere die Abfrage, sodass sie andere Ausschnitte der Daten abbildet. Beantworte auf diese Weise folgende Fragen:  
1) Was machen die reichsten Beziehungssuchenden beruflich?  
2) Wie viele Raucher gibt es unter den Veganern unter 25?  
WICHTIG: Behalte den ```LIMIT 20``` Befehl am Ende bei, sonst könnte dem Notebook der Arbeitsspeicher ausgehen wenn zehntausende Datensätze im Browser anzuzeigen wären.

---

## 5. Verbindung zu APIs herstellen<a id="kapitel5"/>

Für eine REST-API benötigen wir kein SQL - hier werden ggf. Filter in der URL (den "Link") mit angegeben. Wie das jeweils funktioniert, ist der Dokumentation der jeweiligen API zu entnehmen.

Starten wir mit einem einfachen Beispiel: 
>[api.open-notify.org/astros.json](http://api.open-notify.org/astros.json) bietet eine Liste aller derzeit im Weltall lebenden Menschen.   
>Klicke zuerst im Browser auf den Link und sieh dir die Daten an. Dieses Format nennt sich JSON (JavaScript Object Notation, mehr dazu [hier](https://www.w3schools.com/js/js_json_syntax.asp)).  
>Führe im Anschluss den nachfolgenden Code aus, um die Daten mit Python einzulesen.

In [ ]:
import pandas as pd  # Python Paket, das wir auch im weiteren für Datenmanagement verwenden
import requests      # Python Paket, das HTTP-Anfragen (engl. requests) stellen kann

link = "http://api.open-notify.org/astros.json"

# API abfragen
response = requests.get(link)
status = response.status_code

if (status == 200):  # HTTP Statuscode 200 bedeutet "Übertragung erfolgreich"
    print("Abruf erfolgreich!")
else:
    print("Fehler beim Abruf!")

In [ ]:
# Sehen wir uns nun die empfangenen Rohdaten an...
# Welche Struktur erkennst du hier wieder? Wo sind Listen? Wo Dictionaries?
response.json()

In [ ]:
# Nun übernehmen wir die Struktur im Feld "people" (record_path) und erhalten eine Datentabelle
pd.json_normalize(response.json(), record_path="people")

---
## 6. Übungen Datenbank- und API-Abfragen <a id="kapitel6"/>

### <span style="color:#FF5D02;">Aufgabe 6.1: Datenabruf mit SQL</span>

Die Tabelle ```housing_ger``` in der Datenbank ```smi``` auf dem oben verwendeten Datenbankserver enthält ~190MB Daten von zu vermietenden Wohnungen in Deutschland (Stand April 2020). 

Stelle eine Verbindung zum Datenbankserver her und verwende ```SELECT``` Abfragen, um zu ermitteln,   
a) ... was Neubauwohnungen (Spalte "obj_newlyConst") in Berlin (Spalte "obj_regio1") mit 50-55qm (Spalte "obj_livingSpace") in etwa kosten (Spalte "obj_totalRent")  
b) ... wieviele Wohnungen aus dem 19. Jahrhundert (obj_yearConstructed) im Landkreis München (geo_krs) angeboten werden.

### <span style="color:#FF5D02;">Aufgabe 6.2: Datenabruf von APIs</span>
Die API https://corona-api.com gibt unter dem Endpunkt ```/timeline``` die Entwicklung der globalen COVID19-Fälle zurück. Wann wurden die Daten zuletzt aktualisiert?
